In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import imagesize

In [ ]:
# %pip install imagesize

In [ ]:
df = pd.read_csv('test.txt', header=None)

In [ ]:
df.tail()

In [ ]:
df = df[0].str.split(' ', expand=True)

In [ ]:
df.rename(columns={0: "test_id", 1: 'filename', 2: 'test_result', 3: 'data_source'}, inplace=True)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df['full_filepath'] = df.apply(lambda row: os.path.join('test', row.filename), axis=1)

In [ ]:
df.head()

In [ ]:
imagesize.get(df['full_filepath'][0])

In [ ]:
# width, height = imagesize.get("test.png")

In [ ]:
df['x_shape'] = df.apply(lambda row: imagesize.get(row.full_filepath)[0], axis=1)
df['y_shape'] = df.apply(lambda row: imagesize.get(row.full_filepath)[1], axis=1)

In [ ]:
df.head()

In [ ]:
print(df['x_shape'].max())
print(df['y_shape'].max())

In [ ]:
df.describe()

In [ ]:
plt.hist(df['x_shape'])

In [ ]:
plt.hist(df['y_shape'])

In [ ]:
nan_count = df.isna().sum()
print(nan_count)

### TorchXRayVision Testing

In [ ]:
# !pip install torchxrayvision

In [ ]:
import torchxrayvision as xrv
import skimage, torch, torchvision


In [ ]:
import numpy as np
import skimage
import torch
import torchvision
import matplotlib.pyplot as plt
import torchxrayvision as xrv

In [ ]:
model = xrv.baseline_models.chestx_det.PSPNet()


In [ ]:
model

In [ ]:
# Read the image
img_path = "test/MIDRC-RICORD-1C-419639-003251-46647-0.png"  # change this to your image path
img = skimage.io.imread(img_path)

# Normalize the image
img = xrv.datasets.normalize(img, 255)

# Check if the image is already 2D (grayscale), then add a new axis
if img.ndim == 2:
    img = img[None, ...]  # Add a new axis to make it 3D
elif img.ndim == 3:
    # Convert RGB to grayscale if it's not already grayscale
    img = img.mean(2)[None, ...]  # Calculate the mean across the color channels

transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(512)])

img = transform(img)
img = torch.from_numpy(img)

In [ ]:
with torch.no_grad():
    pred = model(img)

In [ ]:
plt.figure(figsize = (26,5))
plt.subplot(1, len(model.targets) + 1, 1)
plt.imshow(img[0], cmap='gray')
for i in range(len(model.targets)):
    plt.subplot(1, len(model.targets) + 1, i+2)
    plt.imshow(pred[0, i])
    plt.title(model.targets[i])
    plt.axis('off')
plt.tight_layout()

In [ ]:
pred = 1 / (1 + np.exp(-pred))  # sigmoid
pred[pred < 0.5] = 0
pred[pred >= 0.5] = 1

In [ ]:
plt.figure(figsize = (26,5))
plt.subplot(1, len(model.targets) + 1, 1)
plt.imshow(img[0], cmap='gray')
for i in range(len(model.targets)):
    plt.subplot(1, len(model.targets) + 1, i+2)
    plt.imshow(pred[0, i])
/    plt.axis('off')
plt.tight_layout()

In [ ]:
left_lung_index = model.targets.index('Left Lung')  # replace 'Left Lung' with the actual target name

left_lung_pred = pred[0, left_lung_index]

print(left_lung_pred)

plt.imshow(pred[0, left_lung_index])

In [ ]:
right_lung_index = model.targets.index('Right Lung')  # replace 'Left Lung' with the actual target name

right_lung_pred = pred[0, right_lung_index]

print(right_lung_pred)

plt.imshow(pred[0, right_lung_index])

In [ ]:
both_lung_pred = np.logical_or(pred[0,left_lung_index], pred[0,right_lung_index])
plt.imshow(both_lung_pred)

In [ ]:
original=img[0]
plt.imshow(img[0], cmap='gray')

In [ ]:
plt.imshow(original*both_lung_pred, cmap='gray')
X = original*both_lung_pred
X[275,200]

In [ ]:
X.shape

In [ ]:
# Relates to the size of the lungs
# if 1 in right_lung_pred[:, 0]:
#     print('Lung pixel found')
# len(right_lung_pred[0])
# leftmost_pixel = -1
# for i in range(len(right_lung_pred[0])):
#     if 1 in right_lung_pred[:, 0]:
#         print('Lung pixel found') 
#         leftmost_pixel = i
        

In [ ]:
x = np.load('etl_output/test/MIDRC-RICORD-1C-419639-000918-78965-0.npy')

In [ ]:
plt.imshow(x)